In [7]:
def set_paths():
    '''
    Deployment should be something like "dirs/dir1/use-cases"
    This function adds to the path "dirs/dir1/use-cases" and "dirs/dir1/"
    :return:
    '''
    import imp
    from os.path import dirname
    import os
    import sys

    USE_CASES = "/var/SP/data/home/asaezco/src/devel2/use-cases"#dirname(os.path.abspath(imp.find_module('churn')[1]))
    sys.path.append("/var/SP/data/home/asaezco/src/devel2/pykhaos")
    if USE_CASES not in sys.path:
        sys.path.append(USE_CASES)
        print("Added '{}' to path".format(USE_CASES))

    # if deployment is correct, this path should be the one that contains "use-cases", "pykhaos", ...
    # FIXME another way of doing it more general?
    DEVEL_SRC = os.path.dirname(USE_CASES)  # dir before use-cases dir
    if DEVEL_SRC not in sys.path:
        sys.path.append(DEVEL_SRC)
        print("Added '{}' to path".format(DEVEL_SRC))
set_paths()

In [8]:
# coding=utf-8

import sys

from common.src.main.python.utils.hdfs_generic import *
import argparse
import os
import sys
import time
from pyspark.sql.functions import (udf,
                                    col,
                                    decode,
                                    when,
                                    lit,
                                    lower,
                                    concat,
                                    translate,
                                    count,
                                    sum as sql_sum,
                                    max as sql_max,
                                    min as sql_min,
                                    avg as sql_avg,
                                    greatest,
                                    least,
                                    isnull,
                                    isnan,
                                    struct, 
                                    substring,
                                    size,
                                    length,
                                    year,
                                    month,
                                    dayofmonth,
                                    unix_timestamp,
                                    date_format,
                                    from_unixtime,
                                    datediff,
                                    to_date, 
                                    desc,
                                    asc,
                                    countDistinct,
                                    row_number,
                                    skewness,
                                    kurtosis,
                                    concat_ws)

from pyspark.sql import Row, DataFrame, Column, Window
from pyspark.sql.types import DoubleType, StringType, IntegerType, DateType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from datetime import datetime
from itertools import chain
import numpy as np
from functools import reduce
from utils_general import *
from pykhaos.utils.date_functions import convert_to_date
from utils_model import *
from utils_fbb_churn import *
from metadata_fbb_churn import *
#from feature_selection_utils import *
import subprocess


In [9]:
# Para que no haga falta recargar el notebook si hacemos un cambio en alguna de las funciones que estamos importando
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# set BDP parameters
def setting_bdp(min_n_executors = 3, max_n_executors = 15, n_cores = 8, executor_memory = "12g", driver_memory="8g",
                   app_name = "Python app", driver_overhead="1g", executor_overhead='3g'):

    MAX_N_EXECUTORS = max_n_executors
    MIN_N_EXECUTORS = min_n_executors
    N_CORES_EXECUTOR = n_cores
    EXECUTOR_IDLE_MAX_TIME = 120
    EXECUTOR_MEMORY = executor_memory
    DRIVER_MEMORY = driver_memory
    N_CORES_DRIVER = 1
    MEMORY_OVERHEAD = N_CORES_EXECUTOR * 2048
    QUEUE = "root.BDPtenants.es.low"
    BDA_CORE_VERSION = "1.0.0"

    SPARK_COMMON_OPTS = os.environ.get('SPARK_COMMON_OPTS', '')
    SPARK_COMMON_OPTS += " --executor-memory %s --driver-memory %s" % (EXECUTOR_MEMORY, DRIVER_MEMORY)
    SPARK_COMMON_OPTS += " --conf spark.shuffle.manager=tungsten-sort"
    SPARK_COMMON_OPTS += "  --queue %s" % QUEUE

    # Dynamic allocation configuration
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.shuffle.service.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.maxExecutors=%s" % (MAX_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.minExecutors=%s" % (MIN_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.executor.cores=%s" % (N_CORES_EXECUTOR)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.executorIdleTimeout=%s" % (EXECUTOR_IDLE_MAX_TIME)
    # SPARK_COMMON_OPTS += " --conf spark.ui.port=58235"
    SPARK_COMMON_OPTS += " --conf spark.port.maxRetries=100"
    SPARK_COMMON_OPTS += " --conf spark.app.name='%s'" % (app_name)
    SPARK_COMMON_OPTS += " --conf spark.submit.deployMode=client"
    SPARK_COMMON_OPTS += " --conf spark.ui.showConsoleProgress=true"
    SPARK_COMMON_OPTS += " --conf spark.sql.broadcastTimeout=1200"
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.memoryOverhead={}".format(executor_overhead)
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.driverOverhead={}".format(driver_overhead)

    BDA_ENV = os.environ.get('BDA_USER_HOME', '')

    # Attach bda-core-ra codebase
    SPARK_COMMON_OPTS+=" --files {}/scripts/properties/red_agent/nodes.properties,{}/scripts/properties/red_agent/nodes-de.properties,{}/scripts/properties/red_agent/nodes-es.properties,{}/scripts/properties/red_agent/nodes-ie.properties,{}/scripts/properties/red_agent/nodes-it.properties,{}/scripts/properties/red_agent/nodes-pt.properties,{}/scripts/properties/red_agent/nodes-uk.properties".format(*[BDA_ENV]*7)

    os.environ["SPARK_COMMON_OPTS"] = SPARK_COMMON_OPTS
    os.environ["PYSPARK_SUBMIT_ARGS"] = "%s pyspark-shell " % SPARK_COMMON_OPTS
    
def get_spark_session(app_name="default name", log_level='INFO', min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    HOME_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src")
    if HOME_SRC not in sys.path:
        sys.path.append(HOME_SRC)


    setting_bdp(app_name=app_name, min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores, executor_memory = executor_memory, driver_memory=driver_memory)
    from common.src.main.python.utils.hdfs_generic import run_sc
    sc, spark, sql_context = run_sc(log_level=log_level)


    return sc, spark, sql_context

def initialize(app_name, min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    import time
    start_time = time.time()

    print("_initialize spark")
    #import pykhaos.utils.pyspark_configuration as pyspark_config
    sc, spark, sql_context = get_spark_session(app_name=app_name, log_level="OFF", min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores,
                             executor_memory = executor_memory, driver_memory=driver_memory)
    print("Ended spark session: {} secs | default parallelism={}".format(time.time() - start_time,
                                                                         sc.defaultParallelism))
    return spark

In [11]:
spark = initialize("FBB Churn Reasons ",executor_memory = "32g",min_n_executors = 6,max_n_executors = 15)


_initialize spark
Ended spark session: 31.4715080261 secs | default parallelism=2


In [13]:
df_mob = spark.read.load('/data/udf/vf_es/churn/churn_reasons/churn_reasons_NIF_MOB')

In [14]:
df_fbb = spark.read.load('/data/udf/vf_es/churn/churn_reasons/churn_reasons_NIF_FBB')

In [15]:
df_mob.printSchema()

root
 |-- msisdn: string (nullable = true)
 |-- num_cliente: string (nullable = true)
 |-- nif_cliente: string (nullable = true)
 |-- max_val_pen: float (nullable = true)
 |-- max_val_serv: float (nullable = true)
 |-- max_val_bill: float (nullable = true)
 |-- max_val_spin: float (nullable = true)
 |-- max_val_use: float (nullable = true)
 |-- max_val_inter: float (nullable = true)
 |-- service: string (nullable = true)
 |-- training_closing_date: string (nullable = true)
 |-- closing_day: string (nullable = true)



In [21]:
print'Numero de servicios: {}'.format(df_mob.count())
print'Numero de NIFs: {}'.format(df_mob.select('nif_cliente').distinct().count())

Numero de servicios: 5976391
Numero de NIFs: 3556494


In [51]:
from pyspark.sql.functions import mean, countDistinct
aggr_mob = df_mob.groupby('NIF_CLIENTE').agg(mean('max_val_pen').alias('max_val_pen_mob'),mean('max_val_serv').alias('max_val_serv_mob'),mean('max_val_bill').alias('max_val_bill_mob'),mean('max_val_spin').alias('max_val_spin_mob'),mean('max_val_use').alias('max_val_use_mob'),mean('max_val_inter').alias('max_val_inter_mob'), countDistinct('msisdn').alias('NUM_MSISDN_MOB'))

In [30]:
aggr_mob.printSchema()

root
 |-- NIF_CLIENTE: string (nullable = true)
 |-- max_val_pen_mob: double (nullable = true)
 |-- max_val_serv_mob: double (nullable = true)
 |-- max_val_bill_mob: double (nullable = true)
 |-- max_val_spin_mob: double (nullable = true)
 |-- max_val_use_mob: double (nullable = true)
 |-- max_val_inter_mob: double (nullable = true)
 |-- NUM_MSISDN: long (nullable = false)



In [34]:
df_mob.columns

['msisdn',
 'num_cliente',
 'nif_cliente',
 'max_val_pen',
 'max_val_serv',
 'max_val_bill',
 'max_val_spin',
 'max_val_use',
 'max_val_inter',
 'service',
 'training_closing_date',
 'closing_day']

In [31]:
aggr_mob.show(10)

+-----------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+----------+
|NIF_CLIENTE|    max_val_pen_mob|   max_val_serv_mob|   max_val_bill_mob|  max_val_spin_mob|    max_val_use_mob| max_val_inter_mob|NUM_MSISDN|
+-----------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+----------+
|  31506457K| 0.5350459218025208| 0.4168987274169922| 0.7936984896659851| 0.550049364566803|                0.0|0.5512686967849731|         4|
|  41401205F| 0.5679730772972107|0.40413597226142883|0.43614208698272705|0.5144904255867004|  0.560025155544281|0.5389700531959534|         2|
|  68587965U| 0.5679730772972107| 0.5482891798019409|0.42269352078437805|0.5348342061042786| 0.5302178263664246|0.5512905120849609|         2|
|  41177586O|0.48807236552238464|                0.0|0.42572513222694397|0.5589956641197205| 0.5166989490389824|0.3765047788619995|         4|

In [50]:
from pyspark.sql.functions import mean, countDistinct
aggr_fbb = df_fbb.groupby('NIF_CLIENTE').agg(mean('max_val_pen').alias('max_val_pen_fbb'),mean('max_val_serv').alias('max_val_serv_fbb'),mean('max_val_bill').alias('max_val_bill_fbb'),mean('max_val_spin').alias('max_val_spin_fbb'),mean('max_val_use').alias('max_val_use_fbb'),mean('max_val_inter').alias('max_val_inter_fbb'), countDistinct('msisdn').alias('NUM_MSISDN_FBB'))

In [52]:
aggr_fbb.printSchema()

root
 |-- NIF_CLIENTE: string (nullable = true)
 |-- max_val_pen_fbb: double (nullable = true)
 |-- max_val_serv_fbb: double (nullable = true)
 |-- max_val_bill_fbb: double (nullable = true)
 |-- max_val_spin_fbb: double (nullable = true)
 |-- max_val_use_fbb: double (nullable = true)
 |-- max_val_inter_fbb: double (nullable = true)
 |-- NUM_MSISDN_FBB: long (nullable = false)



In [22]:
print'Numero de servicios: {}'.format(df_fbb.count())
print'Numero de NIFs: {}'.format(df_fbb.select('nif_cliente').distinct().count())

Numero de servicios: 2338420
Numero de NIFs: 2294177


In [41]:
aggr_fbb.count()

2294177

In [42]:
aggr_mob.count()

3556494

In [43]:
t = aggr_fbb.join(aggr_mob, ['NIF_CLIENTE'], 'inner')

In [44]:
t.count()

2131623

In [45]:
t.select('nif_cliente').distinct().count()

2131623

In [102]:
t_full = aggr_fbb.join(aggr_mob, ['NIF_CLIENTE'], 'full')

In [47]:
t_full.count()

3719048

In [103]:
t_full.columns

['NIF_CLIENTE',
 'max_val_pen_fbb',
 'max_val_serv_fbb',
 'max_val_bill_fbb',
 'max_val_spin_fbb',
 'max_val_use_fbb',
 'max_val_inter_fbb',
 'NUM_MSISDN_FBB',
 'max_val_pen_mob',
 'max_val_serv_mob',
 'max_val_bill_mob',
 'max_val_spin_mob',
 'max_val_use_mob',
 'max_val_inter_mob',
 'NUM_MSISDN_MOB']

In [57]:
t_full.select('NIF_CLIENTE','max_val_pen_fbb','max_val_pen_mob','NUM_MSISDN_MOB', 'NUM_MSISDN_FBB').show(10)

+-----------+------------------+-------------------+--------------+--------------+
|NIF_CLIENTE|   max_val_pen_fbb|    max_val_pen_mob|NUM_MSISDN_MOB|NUM_MSISDN_FBB|
+-----------+------------------+-------------------+--------------+--------------+
|  00016272Y|               0.0| 0.5679730772972107|             1|             1|
|  00016910J|0.5760828256607056| 0.5091647744178772|             5|             1|
|  00025778W|               0.0| 0.5679730772972107|             1|             1|
|  00029524J|              null| 0.4991200268268585|             2|          null|
|  00043834X|              null| 0.5366258025169373|             2|          null|
|  00050353W|              null| 0.4964126646518707|             2|          null|
|  00054330G|              null|0.46673980355262756|             1|          null|
|  00057674M|0.4992365539073944| 0.4985930919647217|             1|             1|
|  00059200K|               0.0|               null|          null|             1|
|  0

In [104]:
fill_cols = ['max_val_pen_fbb','max_val_serv_fbb','max_val_bill_fbb','max_val_spin_fbb','max_val_use_fbb','max_val_inter_fbb','NUM_MSISDN_FBB','max_val_pen_mob','max_val_serv_mob','max_val_bill_mob','max_val_spin_mob','max_val_use_mob','max_val_inter_mob','NUM_MSISDN_MOB']

In [105]:
for name_ in fill_cols:
    t_full = t_full.na.fill({name_ : 0.0})

In [63]:
t_full.select('NIF_CLIENTE','max_val_pen_fbb','max_val_pen_mob','NUM_MSISDN_MOB', 'NUM_MSISDN_FBB').show(10)

+-----------+------------------+-------------------+--------------+--------------+
|NIF_CLIENTE|   max_val_pen_fbb|    max_val_pen_mob|NUM_MSISDN_MOB|NUM_MSISDN_FBB|
+-----------+------------------+-------------------+--------------+--------------+
|  00016272Y|               0.0| 0.5679730772972107|             1|             1|
|  00016910J|0.5760828256607056| 0.5091647744178772|             5|             1|
|  00025778W|               0.0| 0.5679730772972107|             1|             1|
|  00029524J|               0.0| 0.4991200268268585|             2|             0|
|  00043834X|               0.0| 0.5366258025169373|             2|             0|
|  00050353W|               0.0| 0.4964126646518707|             2|             0|
|  00054330G|               0.0|0.46673980355262756|             1|             0|
|  00057674M|0.4992365539073944| 0.4985930919647217|             1|             1|
|  00059200K|               0.0|                0.0|             0|             1|
|  0

In [106]:
names = ['pen', 'serv', 'bill', 'use', 'inter', 'spin']
for name_ in names:
    t_full = t_full.withColumn('max_val_' + name_, (col('max_val_' + name_ + '_fbb')*col('NUM_MSISDN_FBB') + col('max_val_' + name_ + '_mob')*col('NUM_MSISDN_MOB'))/((col('NUM_MSISDN_MOB') + col('NUM_MSISDN_FBB'))))

In [70]:
t_full.count()

3719048

In [107]:
t_full.columns

['NIF_CLIENTE',
 'max_val_pen_fbb',
 'max_val_serv_fbb',
 'max_val_bill_fbb',
 'max_val_spin_fbb',
 'max_val_use_fbb',
 'max_val_inter_fbb',
 'NUM_MSISDN_FBB',
 'max_val_pen_mob',
 'max_val_serv_mob',
 'max_val_bill_mob',
 'max_val_spin_mob',
 'max_val_use_mob',
 'max_val_inter_mob',
 'NUM_MSISDN_MOB',
 'max_val_pen',
 'max_val_serv',
 'max_val_bill',
 'max_val_use',
 'max_val_inter',
 'max_val_spin']

In [108]:
def getTopNVals(spark, df, filtered_cols, n_feats):
    #Function to get the scores of the categories ordered
    for col_name in filtered_cols:
        df = df.withColumn(col_name, col(col_name).cast('float'))
    for i in range (0,n_feats):
        name_val = 'top{}_val'.format(i)
        df = df.withColumn(name_val, sort_array(array([col(x) for x in filtered_cols]), asc=False)[i])
    return(df)

In [109]:
def getTopNFeatsK2(spark, df, n_feats, filtered_cols):
    #Function to create the reasons ranking
    from pyspark.sql.types import DoubleType, StringType
    df2 = df   
    modify_values_udf = udf(modify_values2, StringType())
    for i in range (0,n_feats):
        name_val = 'top{}_val'.format(i)
        name_feat = 'top{}_feat'.format(i)
        df =  df\
        .withColumn(name_feat, modify_values_udf(array(df2.columns[-len(filtered_cols)-n_feats:-n_feats]), name_val, array(filtered_cols)))
 
    for i in range (0,n_feats):
        name_col = 'top{}_reason'.format(i)
        name_feat = 'top{}_feat'.format(i)
        name_val = 'top{}_val'.format(i)
        df =  df\
        .withColumn(name_col, when(df[name_val] > 0, df[name_feat]).otherwise('-'))           
    return(df)

In [110]:
def modify_values2(r, max_col, filtered_cols): 
    l = []
    for i in range(len(filtered_cols)):
        if r[i]== max_col:
            l.append(reasons[i])
    return l

In [111]:
from pyspark.sql.functions import *
reasons = ['Penalization','Engagement','Billing','Spinner', 'Use', 'Interactions']
score_cols_2 = ['max_val_pen','max_val_serv','max_val_bill','max_val_spin', 'max_val_use', 'max_val_inter']
df_ord = getTopNVals(spark, t_full, score_cols_2, 6)

In [112]:
df_final = getTopNFeatsK2(spark, df_ord, 6, filtered_cols = score_cols_2)

In [96]:
import pyspark.sql.functions

In [113]:
df_final.printSchema()

root
 |-- NIF_CLIENTE: string (nullable = true)
 |-- max_val_pen_fbb: double (nullable = false)
 |-- max_val_serv_fbb: double (nullable = false)
 |-- max_val_bill_fbb: double (nullable = false)
 |-- max_val_spin_fbb: double (nullable = false)
 |-- max_val_use_fbb: double (nullable = false)
 |-- max_val_inter_fbb: double (nullable = false)
 |-- NUM_MSISDN_FBB: long (nullable = true)
 |-- max_val_pen_mob: double (nullable = false)
 |-- max_val_serv_mob: double (nullable = false)
 |-- max_val_bill_mob: double (nullable = false)
 |-- max_val_spin_mob: double (nullable = false)
 |-- max_val_use_mob: double (nullable = false)
 |-- max_val_inter_mob: double (nullable = false)
 |-- NUM_MSISDN_MOB: long (nullable = true)
 |-- max_val_pen: float (nullable = true)
 |-- max_val_serv: float (nullable = true)
 |-- max_val_bill: float (nullable = true)
 |-- max_val_use: float (nullable = true)
 |-- max_val_inter: float (nullable = true)
 |-- max_val_spin: float (nullable = true)
 |-- top0_val: float 

In [114]:
df_final.select(name_cols).show(10)

+--------------+--------------+--------------+--------------+--------------+--------------+
|   top0_reason|   top1_reason|   top2_reason|   top3_reason|   top4_reason|   top5_reason|
+--------------+--------------+--------------+--------------+--------------+--------------+
|         [Use]|     [Spinner]|     [Billing]|[Penalization]|[Interactions]|  [Engagement]|
|     [Billing]|     [Spinner]|  [Engagement]|[Interactions]|[Penalization]|         [Use]|
|[Interactions]|         [Use]|     [Spinner]|     [Billing]|[Penalization]|  [Engagement]|
|  [Engagement]|         [Use]|     [Billing]|[Interactions]|     [Spinner]|[Penalization]|
|[Penalization]|[Interactions]|     [Billing]|  [Engagement]|         [Use]|     [Spinner]|
|     [Spinner]|  [Engagement]|     [Billing]|[Interactions]|[Penalization]|             -|
|  [Engagement]|[Interactions]|[Penalization]|     [Billing]|             -|             -|
|[Interactions]|         [Use]|     [Spinner]|[Penalization]|     [Billing]|  [E

In [101]:
name_cols = ['top'+ str(i) + '_reason' for i in range(0,6)]
df_final.printSchema()

root
 |-- NIF_CLIENTE: string (nullable = true)
 |-- max_val_pen_fbb: double (nullable = false)
 |-- max_val_serv_fbb: double (nullable = false)
 |-- max_val_bill_fbb: double (nullable = false)
 |-- max_val_spin_fbb: double (nullable = false)
 |-- max_val_use_fbb: double (nullable = false)
 |-- max_val_inter_fbb: double (nullable = false)
 |-- NUM_MSISDN_FBB: long (nullable = true)
 |-- max_val_pen_mob: double (nullable = false)
 |-- max_val_serv_mob: double (nullable = false)
 |-- max_val_bill_mob: double (nullable = false)
 |-- max_val_spin_mob: double (nullable = false)
 |-- max_val_use_mob: double (nullable = false)
 |-- max_val_inter_mob: double (nullable = false)
 |-- NUM_MSISDN_MOB: long (nullable = true)
 |-- max_val_pen_final: double (nullable = true)
 |-- max_val_serv_final: double (nullable = true)
 |-- max_val_bill_final: double (nullable = true)
 |-- max_val_use_final: double (nullable = true)
 |-- max_val_inter_final: double (nullable = true)
 |-- max_val_spin_final: doub

In [17]:
aggr_f = df_mob.union(df_fbb)

In [19]:
df.count()

8314811

In [20]:
df.select('nif_cliente').distinct().count()

3719048